In [40]:
import nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re


import nltk
nltk.download('punkt')

import warnings
warnings.filterwarnings('ignore')

# import data
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
# Load all the reviews from the dataset
data = pd.read_excel("/content/drive/MyDrive/Exam NLP/test_data_output.xlsx")
data.head()

,review,sentiment,cleaned_text
0,one of the other reviewers has mentioned that ...,positive,one review mention watch oz episod hook right ...
1,a wonderful little production. the filming tec...,positive,wonder littl product film techniqu unassum old...
2,i thought this was a wonderful way to spend ti...,positive,thought wonder way spend time hot summer weeke...
3,basically there's a family where a little boy ...,negative,basic famili littl boy jake think zombi closet...
4,"petter mattei's ""love in the time of money"" is...",positive,petter mattei love time money visual stun film...


In [42]:


# Train Word2Vec model on the entire dataset
sentences = [word_tokenize(review) for review in data['cleaned_text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)


In [43]:

# Function to get sentence vector using Word2Vec
def get_sentence_vector(sentence_tokens):
    # Initialize an empty vector
    sentence_vector = np.zeros(word2vec_model.vector_size)

    # Calculate the sum of word vectors
    num_tokens_found = 0
    for token in sentence_tokens:
        if token in word2vec_model.wv:
            sentence_vector += word2vec_model.wv[token]
            num_tokens_found += 1

    # Calculate the average of word vectors
    if num_tokens_found > 0:
        sentence_vector /= num_tokens_found

    return sentence_vector

# Get vector representation for each sentence in the dataset
data['sentence_vector'] = [get_sentence_vector(tokens) for tokens in sentences]


In [44]:

# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['sentence_vector'], data['sentiment'], test_size=0.2, random_state=42)



In [45]:
# Train the SVM classifier on Word2Vec vectors
from sklearn.svm import SVC
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train.tolist(), y_train)



SVC(kernel='linear')

In [46]:
# Make predictions on the test set
y_pred = svm_classifier.predict(X_test.tolist())


In [47]:
from sklearn.metrics import accuracy_score, classification_report
# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy (Word2Vec approach):", accuracy)
print(classification_report(y_test, y_pred))


Accuracy (Word2Vec approach): 0.8587881822734101
              precision    recall  f1-score   support

    negative       0.86      0.85      0.86       985
    positive       0.85      0.87      0.86      1012

    accuracy                           0.86      1997
   macro avg       0.86      0.86      0.86      1997
weighted avg       0.86      0.86      0.86      1997



The Accuracy for SVC using word2cec is 86%